In [1]:
import pandas as pd
import os

In [ ]:
# Directorio de los datos
data_dir = 'data'
#Lista de dataframes
dfs = []
#Inicializar un dataframe vacio
Data = pd.DataFrame(columns=['FECHA', 'Total_Importaciones','Importaciones_Registradas'])
#Algunos archivos que generan excepciones
different = ['Julio_2019.csv','Diciembre_2019.csv','Noviembre_2019.csv','Mayo_2019.csv']
##Archivos que no importan
exclude = ['DATA_SERIE_IMPORTACIONES.csv','DATA_IMPORTACIONES.csv']

# Logica dentro del directorio
for file_name in os.listdir(data_dir):
    #Crear dataframe con las columnas requeridas
    data = pd.DataFrame(columns=['Total_Importaciones', 'Importaciones_Registradas', 'FECHA'])
    
    #Caso general
    if file_name.endswith('.csv') and file_name not in exclude and file_name not in different:
        #Leer el dataframe, en caso de error, intentar otro tipo de decodificacion
        try:
            df = pd.read_csv(os.path.join(data_dir, file_name), encoding='ISO-8859-1')
        except (pd.errors.ParserError):
            df = pd.read_csv(os.path.join(data_dir, file_name), sep=';')
    ##Para archivos especiales:
    elif file_name.endswith('.csv') and file_name in different:
        df = pd.read_csv(os.path.join(data_dir, file_name), sep=';', encoding='ISO-8859-1')
        
    # convertir'BASEIVA' a una columna numerica
    df['BASEIVA'] = pd.to_numeric(df['BASEIVA'], errors='coerce')

    ##extraer columnas requeridas
    total_importaciones = df['BASEIVA'].sum()
    importaciones_registradas = df['BASEIVA'].size
    fecha = df['FECH'].values[1]

    #Agregar los datos a la data del ciclo
    row = {'Total_Importaciones': total_importaciones, 'Importaciones_Registradas': importaciones_registradas, 'FECHA': fecha}
    data = data.append(row,ignore_index=True)

    #Combinar la data  actual con la general
    Data = pd.concat([Data, data])
            
    dfs.append(df)

In [9]:
Data

,FECHA,Total_Importaciones,Importaciones_Registradas
0,2002,13222499073569,283768
0,1603,11179414934045,227234
0,1710,11080457854443,280111
0,1610,10299136633030,255803
0,2209,28112168932365,322998
...,...,...,...
0,1202,7791626495895,223705
0,1607,9530037804839,228446
0,1309,9411042635870,261757
0,1209,8046037072014,244950


In [10]:
Data.to_csv('data/DATA_SERIE_IMPORTACIONES.csv',index=False)

In [ ]:
# Group the data by date and sum the BASEIVA column
sum_df = combined_df.groupby('FECH')['BASEIVA'].sum().reset_index()

# Count the total rows for each unique 'FECH'
count_df = combined_df.groupby('FECH').size().reset_index(name='total_rows')

# Merge the two DataFrames
result_df = pd.merge(sum_df, count_df, on='FECH')

# Rename the columns
result_df = result_df.rename(columns={'FECH': 'date', 'BASEIVA': 'total'})

# Reorder the columns
result_df = result_df[['date', 'total', 'total_rows']]

In [ ]:
result_df.head()

In [ ]:
# Group the data by year and month and sum the BASEIVA column
time_series = combined_df.groupby(['Year', 'Month']).sum()

# Reset the index to convert the multi-level index to columns
time_series = time_series.reset_index()

# Combine the Year and Month columns into a single date column
time_series['Date'] = pd.to_datetime(time_series['Year'] + time_series['Month'], format='%Y%B')

# Drop the Year and Month columns
time_series = time_series.drop(['Year', 'Month'], axis=1)